In [57]:
import torch
from torch import nn
from torchmetrics import Accuracy, AUROC, ConfusionMatrix
from torchvision.models import resnet50
from datasets.evlp_xray import EVLPDataset
from tasks.binary_classification import BinaryClassificationTask
from finetune_evlp import load_pretrained

pretrained_model_path = "C:/Users/chaob/OneDrive - University of Toronto/Documents/EVLP X-ray Imaging Project/evlp_xray_cv/saved_models/pretrain_nihcxr/epoch=15-step=139199.ckpt"
finetuned_model_path = "C:/Users/chaob/OneDrive - University of Toronto/Documents/EVLP X-ray Imaging Project/evlp_xray_cv/saved_models/finetune_evlp/finetune_head_n=95_BinarizeThreshold1/lightning_logs/version_1/checkpoints/epoch=6-step=552.ckpt"
data_dir = "C:/Users/chaob/OneDrive - University of Toronto/Documents/EVLP X-ray Imaging Project/EVLP CXR/Double/Main"

In [60]:
# Re-create the PyTorch Lightning Module and load it from the saved dictionary
model, frozen_feature_extractor = load_pretrained(model_backbone='resnet50',
                                                  pretrain_path=pretrained_model_path,
                                                  pretrain_was_multilabel=False,
                                                  pretrain_num_labels=1,
                                                  finetune_num_labels=1,
                                                  finetune_all=False)
task = BinaryClassificationTask.load_from_checkpoint(finetuned_model_path,
                                                     model=model,
                                                     frozen_feature_extractor=frozen_feature_extractor)
task.eval()

c:\users\chaob\anaconda3\envs\pytorch\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


BinaryClassificationTask(
  (model): Linear(in_features=2048, out_features=1, bias=True)
  (frozen_feature_extractor): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_ru

In [61]:
val_dataset = EVLPDataset(data_dir, split='val', binarize=True, aggregate_labels=True, aggregate_regions=True)
val_accuracy = Accuracy()
val_auroc = AUROC()

In [62]:
ypreds = []
labels = []
for image, label in val_dataset:
    with torch.no_grad():
        ypred = task(image.unsqueeze(dim=0))
    ypred = torch.sigmoid(ypred)
    ypreds.append(ypred)
    labels.append(label.round().long())
ypreds = torch.stack(ypreds).reshape(-1)
labels = torch.stack(labels)

In [63]:
val_accuracy.update(ypreds, labels)
val_auroc.update(ypreds, labels)

In [64]:
val_accuracy.compute()

tensor(0.8667)

In [65]:
val_auroc.compute()

c:\users\chaob\anaconda3\envs\pytorch\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)


tensor(0.)

In [66]:
ypreds

tensor([0.4865, 0.5692, 0.6423, 0.7658, 0.5076, 0.4753, 0.6077, 0.5949, 0.7501,
        0.7848, 0.6753, 0.6167, 0.7329, 0.5340, 0.5885])

In [67]:
labels

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])